In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys  
sys.path.insert(0, '/Users/yingshuyang/pythonfiles/TransferMatrixMethod/Absorption_Calculation_GUI')
import absorption_calculation_class as cla2
from scipy.interpolate import interp1d
from math import pi
from scipy.optimize import minimize

In [2]:



## =============================================================================
## 1.Absorption calculation for a single frequency
## =============================================================================




##########################################################################
frequency       = 374.74e12    #THz
omega_single    = 2*pi*frequency
f_omega_single  = 10   #the freq amp we set manully



eps_Air         = 1   
eps_1           = cla2.epsilon(3.3,	4.8)             #NiFe(Py)
eps_2           = cla2.epsilon(0.57,8.0776)                  # Pt       
eps_3           = cla2.epsilon(1.78,	0.0000049)                  #Sub(Sapphire)
eps_4           = cla2.epsilon(1,	0)                      #quartz
eps_5           = cla2.epsilon(1,	0)               #whatever
epsilon_optical = [eps_Air*8.85e-12,eps_1*8.85e-12,eps_2*8.85e-12,eps_3*8.85e-12,eps_4*8.85e-12,eps_5*8.85e-12]  #permittivity 
mu              = 12.57e-7 



d1 = 3e-9
d2 = 6e-9
d3 = 0.5e-3
d4 = 0
d5 = 0
number_of_thickness = 1
##########################################################################




ZERO            = np.linspace(0,0,number_of_thickness)
LAYER1          = np.linspace(d1,d1,number_of_thickness)
LAYER2          = np.linspace(d2,d1,number_of_thickness)       
LAYER3          = np.linspace(d3,d3,number_of_thickness)         
LAYER4          = np.linspace(d4,d4,number_of_thickness)
LAYER5          = np.linspace(d5,d5,number_of_thickness)
dnew            = np.transpose([ZERO,LAYER1,LAYER2,LAYER3,LAYER4,LAYER5])

change_Pt_thickness = cla2.Change_thickness(f_omega_single,omega_single,mu,epsilon_optical)
Q_heatloss1, Q_heatloss2, Q_heatloss3, Q_heatloss4, Q_heatloss5, Q_income         = change_Pt_thickness.heatloss(dnew,number_of_thickness)
Absorption1, Absorption2, Absorption3, Absorption4, Absorption5, Absorption_total = change_Pt_thickness.absorption(dnew,number_of_thickness)
## sub(quartz)         Pt            Si        Co          Sio2     Total


reflection      = change_Pt_thickness.reflection(dnew,number_of_thickness)
transmission    = change_Pt_thickness.transmission(dnew,number_of_thickness)

Total           = reflection+transmission+Absorption_total
Absorp          = 1-reflection-transmission
r,t             = change_Pt_thickness.trans_reflect(dnew,number_of_thickness)

atran_heat,atrans_in = change_Pt_thickness.transmission_heat(dnew,number_of_thickness)

matrix       = cla2.SpecialMatrix(omega_single,mu,epsilon_optical,dnew[0])
T_0s, T_sinf, T_0inf                     = matrix.Transfer_Matrix()
at        = matrix.Transmission_coeff(T_0inf)#(-0.2126720162058721+0.7503224950023671j)
t_os        = matrix.Transmission_coeff(T_0s)#(-0.258334517143757+0.928450239723685j)
t_sinf        = matrix.Transmission_coeff(T_sinf)
at_noecho =np.multiply(t_os, t_sinf) 

atran = f_omega_single*at/f_omega_single
atran_noecho =f_omega_single*at_noecho/f_omega_single 





print('Py/Pt/Sub:')
print('Absorption_Py = ', Absorption1)
print('Absorption_PtTe2     = ', Absorption2)
print('Absorption_Sub   = ', Absorption3)
print('Absorption_total 0.4462    = ', Absorption_total)
print('Transmission     0.2908   = ',transmission)
print('Refelction       0.2630     = ',reflection)
print('A+T+R             = ',Absorption_total+transmission+reflection)




Py/Pt/Sub:
Absorption_Py =  [0.12337598]
Absorption_PtTe2     =  [0.0578639]
Absorption_Sub   =  [0.00729876]
Absorption_total 0.4462    =  [0.18853864]
Transmission     0.2908   =  [0.15933649]
Refelction       0.2630     =  [0.65212487]
A+T+R             =  [1.]


In [ ]:
import tkinter as tk
import json

def save_data():
    data = {
        "frequency": float(frequency_entry.get()),
        "layer1_thickness": float(layer1_thickness_entry.get()),
        "layer2_thickness": float(layer2_thickness_entry.get()),
        "layer3_thickness": float(layer3_thickness_entry.get()),
        "layer4_thickness": float(layer4_thickness_entry.get()),
        "layer5_thickness": float(layer5_thickness_entry.get()),
        "layer1_refractive_index": [float(layer1_n_entry.get()), float(layer1_k_entry.get())],
        "layer2_refractive_index": [float(layer2_n_entry.get()), float(layer2_k_entry.get())],
        "layer3_refractive_index": [float(layer3_n_entry.get()), float(layer3_k_entry.get())],
        "layer4_refractive_index": [float(layer4_n_entry.get()), float(layer4_k_entry.get())],
        "layer5_refractive_index": [float(layer4_n_entry.get()), float(layer4_k_entry.get())],
    }
    with open('data.json', 'w') as f:
        json.dump(data, f)
    print("Data saved.")

def run_code():
    with open('data.json', 'r') as f:
        data = json.load(f)
    f = open('/Users/yingshuyang/pythonfiles/TransferMatrixMethod/Absorption_Calculation_GUI/Optical_Absorption_GUI/data.json')    
    
    # Get the frequency from the data
    frequency = data['frequency']
    d1 = data['layer1_thickness']
    d2 = data['layer1_thickness']
    d3 = data['layer1_thickness']
    d4 = data['layer1_thickness']
    d5 = data['layer1_thickness']

    n1,k1 = data['layer1_refractive_index']
    n2,k2 = data['layer2_refractive_index']
    n3,k3 = data['layer3_refractive_index']
    n4,k4 = data['layer4_refractive_index']
    n5,k5 = data['layer5_refractive_index']
    ##########################################################################
    omega_single    = 2*pi*frequency
    f_omega_single  = 10   #the freq amp we set manully



    eps_Air         = 1   
    eps_1           = cla2.epsilon(n1,k1)             #NiFe(Py)
    eps_2           = cla2.epsilon(n2,k2)                  # Pt       
    eps_3           = cla2.epsilon(n3,k3)                  #Sub(Sapphire)
    eps_4           = cla2.epsilon(n4,k4)                      #quartz
    eps_5           = cla2.epsilon(n5,k5)                      #quartz
    epsilon_optical = [eps_Air*8.85e-12,eps_1*8.85e-12,eps_2*8.85e-12,eps_3*8.85e-12,eps_4*8.85e-12,eps_4*8.85e-12]  #permittivity 
    mu              = 12.57e-7 




    number_of_thickness = 1
    ##########################################################################




    ZERO            = np.linspace(0,0,number_of_thickness)
    LAYER1          = np.linspace(d1,d1,number_of_thickness)
    LAYER2          = np.linspace(d2,d1,number_of_thickness)       
    LAYER3          = np.linspace(d3,d3,number_of_thickness)         
    LAYER4          = np.linspace(d4,d4,number_of_thickness)
    LAYER5          = np.linspace(d5,d5,number_of_thickness)
    dnew            = np.transpose([ZERO,LAYER1,LAYER2,LAYER3,LAYER4,LAYER5])

    change_Pt_thickness = cla2.Change_thickness(f_omega_single,omega_single,mu,epsilon_optical)
    Q_heatloss1, Q_heatloss2, Q_heatloss3, Q_heatloss4, Q_heatloss5, Q_income         = change_Pt_thickness.heatloss(dnew,number_of_thickness)
    Absorption1, Absorption2, Absorption3, Absorption4, Absorption5, Absorption_total = change_Pt_thickness.absorption(dnew,number_of_thickness)
    ## sub(quartz)         Pt            Si        Co          Sio2     Total


    reflection      = change_Pt_thickness.reflection(dnew,number_of_thickness)
    transmission    = change_Pt_thickness.transmission(dnew,number_of_thickness)

    Total           = reflection+transmission+Absorption_total
    Absorp          = 1-reflection-transmission
    r,t             = change_Pt_thickness.trans_reflect(dnew,number_of_thickness)

    atran_heat,atrans_in = change_Pt_thickness.transmission_heat(dnew,number_of_thickness)

    matrix       = cla2.SpecialMatrix(omega_single,mu,epsilon_optical,dnew[0])
    T_0s, T_sinf, T_0inf                     = matrix.Transfer_Matrix()
    at        = matrix.Transmission_coeff(T_0inf)#(-0.2126720162058721+0.7503224950023671j)
    t_os        = matrix.Transmission_coeff(T_0s)#(-0.258334517143757+0.928450239723685j)
    t_sinf        = matrix.Transmission_coeff(T_sinf)
    at_noecho =np.multiply(t_os, t_sinf) 

    atran = f_omega_single*at/f_omega_single
    atran_noecho =f_omega_single*at_noecho/f_omega_single 

    results = f"Py/Pt/Sub:\nAbsorption_Py = {Absorption1}\nAbsorption_PtTe2 = {Absorption2}\n" \
              f"Absorption_Sub = {Absorption3}\nAbsorption_total = {Absorption_total}"
    result_label.config(text=results)



    print('Py/Pt/Sub:')
    print('Absorption_Py = ', Absorption1)
    print('Absorption_PtTe2     = ', Absorption2)
    print('Absorption_Sub   = ', Absorption3)
    print('Absorption_total 0.4462    = ', Absorption_total)
    print('Transmission     0.2908   = ',transmission)
    print('Refelction       0.2630     = ',reflection)
    print('A+T+R             = ',Absorption_total+transmission+reflection)


    
    
    
    

## =============================================================================
## GUI design
## =============================================================================    
    
root = tk.Tk()
root.title("Absorption Calculation")

# Create labels and entry fields for each input
tk.Label(root, text="Frequency (THz)").grid(row=0, column=0)
frequency_entry = tk.Entry(root)
frequency_entry.grid(row=0, column=1)

tk.Label(root, text="Layer 1 thickness (m)").grid(row=1, column=0)
layer1_thickness_entry = tk.Entry(root)
layer1_thickness_entry.grid(row=1, column=1)

tk.Label(root, text="Layer 2 thickness (m)").grid(row=2, column=0)
layer2_thickness_entry = tk.Entry(root)
layer2_thickness_entry.grid(row=2, column=1)

tk.Label(root, text="Layer 3 thickness (m)").grid(row=3, column=0)
layer3_thickness_entry = tk.Entry(root)
layer3_thickness_entry.grid(row=3, column=1)

tk.Label(root, text="Layer 4 thickness (m)").grid(row=4, column=0)
layer4_thickness_entry = tk.Entry(root)
layer4_thickness_entry.grid(row=4, column=1)


tk.Label(root, text="Layer 5 thickness (m)").grid(row=5, column=0)
layer5_thickness_entry = tk.Entry(root)
layer5_thickness_entry.grid(row=5, column=1)

tk.Label(root, text="Layer 1 n, k").grid(row=6, column=0)
layer1_n_entry = tk.Entry(root)
layer1_n_entry.grid(row=6, column=1)
layer1_k_entry = tk.Entry(root)
layer1_k_entry.grid(row=6, column=2)

tk.Label(root, text="Layer 2 n, k").grid(row=7, column=0)
layer2_n_entry = tk.Entry(root)
layer2_n_entry.grid(row=7, column=1)
layer2_k_entry = tk.Entry(root)
layer2_k_entry.grid(row=7, column=2)

tk.Label(root, text="Layer 3 n, k").grid(row=8, column=0)
layer3_n_entry = tk.Entry(root)
layer3_n_entry.grid(row=8, column=1)
layer3_k_entry = tk.Entry(root)
layer3_k_entry.grid(row=8, column=2)

tk.Label(root, text="Layer 4 n, k").grid(row=9, column=0)
layer4_n_entry = tk.Entry(root)
layer4_n_entry.grid(row=9, column=1)
layer4_k_entry = tk.Entry(root)
layer4_k_entry.grid(row=9, column=2)

tk.Label(root, text="Layer 5 n, k").grid(row=10, column=0)
layer5_n_entry = tk.Entry(root)
layer5_n_entry.grid(row=10, column=1)
layer5_k_entry = tk.Entry(root)
layer5_k_entry.grid(row=10, column=2)

# Create a button to save the data as a JSON file
button_save = tk.Button(root, text="Save", command=save_data)
button_save.grid(row = 11,column=1)

# Create a button to run the code and plot the pulse
button_run = tk.Button(root, text="Run and Plot", command=run_code)
button_run.grid(row = 12,column=1)


result_label = tk.Label(root, text="Click Run and Plot to show results")
result_label.grid(row=13, column=0, columnspan=3)

root.mainloop()

Data saved.
Py/Pt/Sub:
Absorption_Py =  [0.26064635]
Absorption_PtTe2     =  [0.07127065]
Absorption_Sub   =  [1.33771898e-07]
Absorption_total 0.4462    =  [0.33191713]
Transmission     0.2908   =  [0.3262724]
Refelction       0.2630     =  [0.34181047]
A+T+R             =  [1.]
